# Basic Slider integration

In [1]:
from ngsolve import *
from netgen.occ import unit_square
#from netgen.occ import SplineApproximation
from ngsolve.webgui import Draw
from time import sleep

#widget stuff
import ipywidgets as widgets
from ipywidgets import interact, fixed

In [2]:
mesh=Mesh(unit_square.GenerateMesh(maxh=0.1))



Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 1, 'order3d': 1, 'draw_vol': None…

BaseWebGuiScene

In [3]:
fes_def=VectorH1(mesh,order=3,dirichlet="top|left|bottom")
gf_def_top=GridFunction(fes_def)
gf_def_bot=GridFunction(fes_def)

delta_top=Parameter(-0.3)
delta_bot=Parameter(0.3)

gf_def_top.Set((0,delta_top*4*x*(1-x)*y))
gf_def_bot.Set((0,delta_bot*4*x*(1-x)*(1-y)))
Draw(gf_def_top)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

# Fun With Sliders:

#### The problem is that I am not able to deform an already deformed grid... It overrides the previous deformation. Therefor both sliders work but only the upper OR lower boundary gets deformed... (tried a lot of stuff with the "fixed" argument, still won't work the way I want...)

In [4]:
mesh.SetDeformation(gf_def_top)
mesh.SetDeformation(gf_def_bot)
scene=Draw(x,mesh)

def change_grid_layout_1 (delta_val_top,scene):

    gf_def_top.Set((0,delta_val_top*4*x*(1-x)*y))
    #gf_def_bot.Set((0,delta_val_bot*4*x*(1-x)*(1-y)))
    #if delta_val_top != 0:
    mesh.SetDeformation(gf_def_top)
    scene.Redraw()
    #else:
    #mesh.SetDeformation(gf_def_bot)
    #scene.Redraw()
    return;

def change_grid_layout_2 (delta_val_bot,scene):

    #gf_def_top.Set((0,delta_val_top*4*x*(1-x)*y))
    gf_def_bot.Set((0,delta_val_bot*4*x*(1-x)*(1-y)))
    #if delta_val_top != 0:
    #mesh.SetDeformation(gf_def_top)
    #scene.Redraw()
    #else:
    mesh.SetDeformation(gf_def_bot)
    scene.Redraw()
    return;
'''
interact(change_grid_layout_1, delta_val_top=(widgets.FloatSlider(
    value=0,
    min=-0.7,
    max=0.7,
    step=0.05,
    description='Deformation:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)),
delta_val_bot=(widgets.FloatSlider(
    value=0,
    min=-0.7,
    max=0.7,
    step=0.05,
    description='Deformation:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)),
mesh=fixed(mesh))
''' 

interact(change_grid_layout_1, delta_val_top=(widgets.FloatSlider(
    value=0,
    min=-0.7,
    max=0.7,
    step=0.05,
    description='Deformation top:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)),
scene=fixed(scene)
)

interact(change_grid_layout_2,delta_val_bot=(widgets.FloatSlider(
    value=0,
    min=-0.7,
    max=0.7,
    step=0.05,
    description='Deformation bottom:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)),
scene=fixed(scene))

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

interactive(children=(FloatSlider(value=0.0, description='Deformation top:', max=0.7, min=-0.7, step=0.05), Ou…

interactive(children=(FloatSlider(value=0.0, description='Deformation bottom:', max=0.7, min=-0.7, step=0.05),…

<function __main__.change_grid_layout_2(delta_val_bot, scene)>

# Fun with Navier Stokes

In [5]:
# viscosity
nu = 0.001

# timestepping parameters
tau = 0.001
tend = 10.0

## Still having problems with the boundary conditions... If I delete the "right" dirichlet BC the inflow enters from the right insted from the left hand side... I think I'm missing something there...

In [6]:
mesh.Curve(3)
#V = VectorH1(mesh,order=3, dirichlet="bottom|top|left")
V = VectorH1(mesh,order=3, dirichlet="left|top|bottom|right")
Q = H1(mesh,order=2)

X = V*Q

u,p = X.TrialFunction()
v,q = X.TestFunction()

stokes = nu*InnerProduct(grad(u), grad(v))+div(u)*q+div(v)*p - 1e-10*p*q
a = BilinearForm(X, symmetric=True)
a += stokes*dx
a.Assemble()

# nothing here ... right hand side =0
f = LinearForm(X)   
f.Assemble()

# gridfunction for the solution
gfu = GridFunction(X)

# parabolic inflow at inlet:
uin = CoefficientFunction( (3*y*(1.0-y), 0) ) #very close to Re_krit
#uin = CoefficientFunction( (40.0*y*(1.0-y), 0) ) #bigger than Re_krit -> unstable
#uin = CoefficientFunction( (0.4*y*(1.0-y), 0) ) #Re=200 stable but verry slow...
gfu.components[0].Set(uin, definedon=mesh.Boundaries("left"))

# solve Stokes problem for initial conditions:
inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res


# matrix for implicit Euler 
mstar = BilinearForm(X, symmetric=True)
#mstar = BilinearForm(X)
mstar += (u*v + tau*stokes)*dx
mstar.Assemble()
inv = mstar.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky")

# the non-linear term 
conv = BilinearForm(X, nonassemble = True)
conv += (grad(u) * u) * v * dx

In [7]:
Draw(Norm(gfu.components[0]),mesh,"velocity")

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

In [8]:
# for visualization
#Draw (Norm(gfu.components[0]), mesh, "velocity")
#Draw (grad(gfu.components[0]), mesh, "direction")
scene_2=Draw(Norm(gfu.components[0]),mesh,"velocity")
sleep(2)
#Draw(Norm(gfu.components[0]),mesh)
#scene_2=Draw(gfu.components[0],mesh)

count=0
# implicit Euler/explicit Euler splitting method:
t = 0
with TaskManager():
    while t < tend:
        #print ("t=", t, end="\r")

        conv.Apply (gfu.vec, res)
        res.data += a.mat*gfu.vec
        gfu.vec.data -= tau * inv * res    

        t = t + tau
        if(count%25==0):
            scene_2.Redraw()
            sleep(0.05)
        count +=1


WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

In [43]:
help(unit_square)

Help on OCCGeometry in module netgen.libngpy._NgOCC object:

class OCCGeometry(netgen.libngpy._meshing.NetgenGeometry)
 |  Use LoadOCCGeometry to load the geometry from a *.step file.
 |  
 |  Method resolution order:
 |      OCCGeometry
 |      netgen.libngpy._meshing.NetgenGeometry
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  Draw(...)
 |      Draw(self: netgen.libngpy._NgOCC.OCCGeometry) -> None
 |  
 |  GenerateMesh(...)
 |      GenerateMesh(self: netgen.libngpy._NgOCC.OCCGeometry, mp: netgen.libngpy._meshing.MeshingParameters = None, **kwargs) -> netgen.libngpy._meshing.Mesh
 |      
 |      
 |      Meshing Parameters
 |      -------------------
 |      
 |      maxh: float = 1e10
 |        Global upper bound for mesh size.
 |      
 |      grading: float = 0.3
 |        Mesh grading how fast the local mesh size can change.
 |      
 |      meshsizefilename: str = None
 |        Load meshsize from file. Can set local me